# ⚙️ Aula 2 – Paralelismo Avançado e Escalabilidade

## Computação de Alto Desempenho em Python para Engenharia Civil

**Objetivos desta aula:**
- Usar ferramentas modernas de paralelismo em CPU
- Medir escalabilidade e compreender overhead
- Introduzir `joblib` e `numba` para otimização
- Aplicar conceitos em problemas de engenharia

---

### 🎯 Revisão da Aula 1

Na aula anterior aprendemos:
- Conceitos de paralelismo, speedup e eficiência
- Limitações do GIL em Python
- Multiprocessing com `ProcessPoolExecutor`
- Aplicações em Monte Carlo e multiplicação de matrizes

### 📈 Foco desta Aula: Escalabilidade

**Strong Scaling**: Problema fixo, aumentar recursos  
**Weak Scaling**: Problema cresce proporcionalmente aos recursos

In [ ]:
# Import Required Libraries
import time
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp
from joblib import Parallel, delayed
import cProfile
import pstats
import io
from functools import wraps

# Tentar importar numba (pode não estar disponível)
try:
    import numba
    from numba import jit, njit, prange
    NUMBA_AVAILABLE = True
    print(f"✅ Numba {numba.__version__} disponível")
except ImportError:
    NUMBA_AVAILABLE = False
    print("⚠️  Numba não disponível - instale com: pip install numba")

# Configure matplotlib
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
plt.rcParams['figure.figsize'] = (12, 8)

print(f"🖥️  Sistema: {mp.cpu_count()} núcleos de CPU")
print(f"📊 NumPy: {np.__version__}")

## 1. Ferramentas de Medição de Performance

Antes de otimizar, precisamos medir com precisão!

In [ ]:
def time_function(func):
    """Decorator para medir tempo de execução"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__name__}: {end - start:.4f}s")
        return result
    return wrapper

def profile_function(func, *args, **kwargs):
    """Profile detalhado de uma função"""
    pr = cProfile.Profile()
    pr.enable()
    result = func(*args, **kwargs)
    pr.disable()
    
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats(10)  # Top 10 funções
    
    print("📊 Profile detalhado:")
    print(s.getvalue())
    return result

# Exemplo de função para testar
@time_function
def compute_intensive_task(n):
    """Tarefa computacionalmente intensiva"""
    total = 0
    for i in range(n):
        total += i ** 2
    return total

# Teste das ferramentas de medição
print("🔍 Testando ferramentas de medição:")
result = compute_intensive_task(1_000_000)
print(f"Resultado: {result}")

print("\\n📈 Profile detalhado:")
profile_function(compute_intensive_task, 1_000_000)

## 2. Exemplo 4: Integração Numérica com Futures

Vamos implementar integração trapezoidal paralela - fundamental para análises em engenharia.

In [ ]:
def function_to_integrate(x):
    """Função exemplo: f(x) = x²·sin(x) + cos(x)"""
    return x**2 * np.sin(x) + np.cos(x)

def trapezoidal_rule_chunk(args):
    """Integração trapezoidal para um chunk do domínio"""
    start, end, n_points, func = args
    
    x = np.linspace(start, end, n_points)
    y = func(x)
    
    # Regra do trapézio
    h = (end - start) / (n_points - 1)
    integral = h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])
    
    return integral

def integrate_parallel(func, a, b, n_total, n_processes):
    """Integração paralela usando futures"""
    start_time = time.perf_counter()
    
    # Dividir domínio entre processos
    chunk_width = (b - a) / n_processes
    points_per_chunk = n_total // n_processes
    
    with ProcessPoolExecutor(max_workers=n_processes) as executor:
        futures = []
        
        for i in range(n_processes):
            chunk_start = a + i * chunk_width
            chunk_end = chunk_start + chunk_width
            
            # Último chunk pode ter pontos extras
            if i == n_processes - 1:
                chunk_end = b
                points_per_chunk = n_total - i * points_per_chunk
            
            args = (chunk_start, chunk_end, points_per_chunk, func)
            future = executor.submit(trapezoidal_rule_chunk, args)
            futures.append(future)
        
        # Coletar resultados
        total_integral = sum(future.result() for future in as_completed(futures))
    
    end_time = time.perf_counter()
    return total_integral, end_time - start_time

def integrate_serial(func, a, b, n_points):
    """Integração serial para comparação"""
    start_time = time.perf_counter()
    
    x = np.linspace(a, b, n_points)
    y = func(x)
    
    h = (b - a) / (n_points - 1)
    integral = h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])
    
    end_time = time.perf_counter()
    return integral, end_time - start_time

# Teste de integração numérica
print("🧮 Integração Numérica Paralela")
print("=" * 40)

# Parâmetros de integração
a, b = 0, 10  # Domínio de integração
n_points = 10_000_000  # Número de pontos

print(f"Integrando f(x) = x²·sin(x) + cos(x) de {a} a {b}")
print(f"Pontos de integração: {n_points:,}")

# Comparar diferentes números de processos
process_counts = [1, 2, 4, mp.cpu_count()]
results = []

for n_proc in process_counts:
    if n_proc == 1:
        # Usar versão serial
        integral, exec_time = integrate_serial(function_to_integrate, a, b, n_points)
        method = "Serial"
    else:
        # Usar versão paralela
        integral, exec_time = integrate_parallel(function_to_integrate, a, b, n_points, n_proc)
        method = f"{n_proc} proc"
    
    results.append({
        'processes': n_proc,
        'integral': integral,
        'time': exec_time,
        'method': method
    })
    
    print(f"{method:>10}: Integral = {integral:.6f}, Tempo = {exec_time:.3f}s")

# Calcular speedups
serial_time = results[0]['time']
print(f"\\n📊 Análise de Speedup:")
for result in results[1:]:
    speedup = serial_time / result['time']
    efficiency = speedup / result['processes']
    print(f"  {result['processes']} processos: Speedup = {speedup:.2f}x, Eficiência = {efficiency:.2f}")

# Visualizar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de tempo de execução
processes = [r['processes'] for r in results]
times = [r['time'] for r in results]

ax1.plot(processes, times, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Número de Processos')
ax1.set_ylabel('Tempo de Execução (s)')
ax1.set_title('Tempo vs Número de Processos')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(processes)

# Gráfico de speedup
speedups = [serial_time / t for t in times]
ideal_speedup = processes

ax2.plot(processes, speedups, 'go-', label='Speedup Real', linewidth=2, markersize=8)
ax2.plot(processes, ideal_speedup, 'r--', label='Speedup Ideal', linewidth=2)
ax2.set_xlabel('Número de Processos')
ax2.set_ylabel('Speedup')
ax2.set_title('Speedup Real vs Ideal')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xticks(processes)

plt.tight_layout()
plt.show()

print("\\n💡 Aplicações em Engenharia:")
print("• Integração de cargas distribuídas")
print("• Cálculo de momentos e centroides")
print("• Análise de espectros de resposta")
print("• Integração de equações diferenciais")

## 3. Biblioteca Joblib - Paralelização Simples

Joblib oferece uma interface mais simples para paralelização, especialmente útil para loops.

In [ ]:
def simulate_beam_response(params):
    """
    Simula resposta de uma viga para diferentes parâmetros
    Útil para análise paramétrica em engenharia estrutural
    """
    E, I, L, P = params  # Módulo, inércia, comprimento, carga
    
    # Deflexão máxima (viga simplesmente apoiada)
    delta_max = (P * L**3) / (48 * E * I)
    
    # Momento máximo
    M_max = P * L / 4
    
    # Tensão máxima (assumindo seção retangular)
    h = (12 * I)**0.5  # Altura assumindo base unitária
    sigma_max = M_max * h / (2 * I)
    
    return {
        'E': E, 'I': I, 'L': L, 'P': P,
        'delta_max': delta_max,
        'M_max': M_max,
        'sigma_max': sigma_max
    }

def linear_regression_worker(data_chunk):
    """Worker para regressão linear em chunk de dados"""
    X, y = data_chunk
    
    # Implementação simples de regressão linear: β = (X'X)⁻¹X'y
    XtX = np.dot(X.T, X)
    Xty = np.dot(X.T, y)
    beta = np.linalg.solve(XtX, Xty)
    
    # Calcular R²
    y_pred = np.dot(X, beta)
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    
    return beta, r_squared

# Exemplo 5: Análise Paramétrica com Joblib
print("🔧 Exemplo 5: Análise Paramétrica de Vigas com Joblib")
print("=" * 55)

# Gerar parâmetros para análise
np.random.seed(42)
n_simulations = 10_000

# Parâmetros estruturais (variação realística)
E_values = np.random.normal(200e9, 20e9, n_simulations)  # Módulo de elasticidade (Pa)
I_values = np.random.uniform(1e-4, 5e-4, n_simulations)  # Momento de inércia (m⁴)
L_values = np.random.uniform(3, 12, n_simulations)       # Comprimento (m)
P_values = np.random.uniform(10e3, 100e3, n_simulations) # Carga (N)

parameters = list(zip(E_values, I_values, L_values, P_values))

print(f"Analisando {n_simulations:,} configurações de vigas...")

# Comparar serial vs joblib
print("\\n⏱️  Comparação de performance:")

# Método serial
start_time = time.perf_counter()
results_serial = [simulate_beam_response(params) for params in parameters]
time_serial = time.perf_counter() - start_time
print(f"Serial:  {time_serial:.3f}s")

# Método joblib (paralelo)
start_time = time.perf_counter()
results_joblib = Parallel(n_jobs=mp.cpu_count())(
    delayed(simulate_beam_response)(params) for params in parameters
)
time_joblib = time.perf_counter() - start_time
print(f"Joblib:  {time_joblib:.3f}s")

speedup = time_serial / time_joblib
print(f"Speedup: {speedup:.2f}x")

# Extrair resultados para análise
deflections = [r['delta_max'] for r in results_joblib]
stresses = [r['sigma_max'] for r in results_joblib]

print(f"\\n📊 Estatísticas dos resultados:")
print(f"  Deflexão máxima: {np.mean(deflections)*1000:.2f} ± {np.std(deflections)*1000:.2f} mm")
print(f"  Tensão máxima:   {np.mean(stresses)/1e6:.2f} ± {np.std(stresses)/1e6:.2f} MPa")

# Exemplo 6: Regressões Lineares Paralelas
print("\\n🧮 Exemplo 6: Regressões Lineares Paralelas com Joblib")
print("=" * 55)

# Gerar dados para múltiplas regressões
n_datasets = 1000
n_points = 1000
n_features = 5

datasets = []
for i in range(n_datasets):
    X = np.random.randn(n_points, n_features)
    X = np.column_stack([np.ones(n_points), X])  # Adicionar intercept
    true_beta = np.random.randn(n_features + 1)
    y = np.dot(X, true_beta) + 0.1 * np.random.randn(n_points)
    datasets.append((X, y))

print(f"Executando {n_datasets} regressões lineares...")
print(f"Cada dataset: {n_points} pontos, {n_features} features")

# Comparar performance
start_time = time.perf_counter()
results_serial = [linear_regression_worker(data) for data in datasets]
time_serial = time.perf_counter() - start_time

start_time = time.perf_counter()
results_parallel = Parallel(n_jobs=mp.cpu_count())(
    delayed(linear_regression_worker)(data) for data in datasets
)
time_parallel = time.perf_counter() - start_time

speedup = time_serial / time_parallel
print(f"\\nSerial:   {time_serial:.3f}s")
print(f"Paralelo: {time_parallel:.3f}s")
print(f"Speedup:  {speedup:.2f}x")

# Análise dos R²
r_squared_values = [r[1] for r in results_parallel]
print(f"\\nR² médio: {np.mean(r_squared_values):.4f} ± {np.std(r_squared_values):.4f}")

print("\\n✅ Joblib oferece:")
print("• Interface simples e intuitiva")
print("• Otimizações automáticas")
print("• Usado internamente pelo scikit-learn")
print("• Ideal para loops embaraçosamente paralelos")

## 4. Numba - Compilação JIT para Aceleração Extrema

Numba oferece speedups dramáticos através de compilação Just-In-Time (JIT).

In [ ]:
if NUMBA_AVAILABLE:
    # Exemplo 7: Multiplicação de Matrizes com Numba
    
    def matrix_mult_python(A, B):
        """Multiplicação de matrizes em Python puro (lento)"""
        rows_A, cols_A = A.shape
        rows_B, cols_B = B.shape
        
        C = np.zeros((rows_A, cols_B))
        for i in range(rows_A):
            for j in range(cols_B):
                for k in range(cols_A):
                    C[i, j] += A[i, k] * B[k, j]
        return C
    
    @jit(nopython=True)
    def matrix_mult_numba_serial(A, B):
        """Multiplicação de matrizes com Numba (serial)"""
        rows_A, cols_A = A.shape
        rows_B, cols_B = B.shape
        
        C = np.zeros((rows_A, cols_B))
        for i in range(rows_A):
            for j in range(cols_B):
                for k in range(cols_A):
                    C[i, j] += A[i, k] * B[k, j]
        return C
    
    @jit(nopython=True, parallel=True)
    def matrix_mult_numba_parallel(A, B):
        """Multiplicação de matrizes com Numba paralelo"""
        rows_A, cols_A = A.shape
        rows_B, cols_B = B.shape
        
        C = np.zeros((rows_A, cols_B))
        for i in prange(rows_A):  # prange = parallel range
            for j in range(cols_B):
                for k in range(cols_A):
                    C[i, j] += A[i, k] * B[k, j]
        return C
    
    print("🚀 Exemplo 7: Multiplicação de Matrizes com Numba")
    print("=" * 50)
    
    # Testar com matrizes pequenas para demonstração
    size = 512
    A = np.random.randn(size, size).astype(np.float64)
    B = np.random.randn(size, size).astype(np.float64)
    
    print(f"Testando multiplicação de matrizes {size}x{size}")
    
    # NumPy (baseline)
    start = time.perf_counter()
    C_numpy = np.dot(A, B)
    time_numpy = time.perf_counter() - start
    print(f"NumPy:              {time_numpy:.4f}s")
    
    # Python puro (apenas para matrizes pequenas)
    if size <= 256:
        start = time.perf_counter()
        C_python = matrix_mult_python(A[:256, :256], B[:256, :256])
        time_python = time.perf_counter() - start
        print(f"Python puro (256²): {time_python:.4f}s")
    
    # Numba serial (primeira execução inclui compilação)
    start = time.perf_counter()
    C_numba_serial = matrix_mult_numba_serial(A, B)
    time_numba_serial_first = time.perf_counter() - start
    print(f"Numba serial (1ª):  {time_numba_serial_first:.4f}s (inclui compilação)")
    
    # Numba serial (segunda execução, já compilado)
    start = time.perf_counter()
    C_numba_serial = matrix_mult_numba_serial(A, B)
    time_numba_serial = time.perf_counter() - start
    print(f"Numba serial (2ª):  {time_numba_serial:.4f}s")
    
    # Numba paralelo
    start = time.perf_counter()
    C_numba_parallel = matrix_mult_numba_parallel(A, B)
    time_numba_parallel_first = time.perf_counter() - start
    print(f"Numba paralelo (1ª): {time_numba_parallel_first:.4f}s (inclui compilação)")
    
    start = time.perf_counter()
    C_numba_parallel = matrix_mult_numba_parallel(A, B)
    time_numba_parallel = time.perf_counter() - start
    print(f"Numba paralelo (2ª): {time_numba_parallel:.4f}s")
    
    # Verificar correção
    numpy_vs_serial = np.allclose(C_numpy, C_numba_serial)
    numpy_vs_parallel = np.allclose(C_numpy, C_numba_parallel)
    
    print(f"\\n🔍 Verificação de resultados:")
    print(f"  NumPy vs Numba serial:   {'✓' if numpy_vs_serial else '✗'}")
    print(f"  NumPy vs Numba paralelo: {'✓' if numpy_vs_parallel else '✗'}")
    
    # Speedups
    speedup_numba_serial = time_numpy / time_numba_serial
    speedup_numba_parallel = time_numpy / time_numba_parallel
    parallel_vs_serial = time_numba_serial / time_numba_parallel
    
    print(f"\\n📈 Speedups:")
    print(f"  Numba serial vs NumPy:     {speedup_numba_serial:.2f}x")
    print(f"  Numba paralelo vs NumPy:   {speedup_numba_parallel:.2f}x")
    print(f"  Numba paralelo vs serial:  {parallel_vs_serial:.2f}x")
    
    # Exemplo adicional: função matemática complexa
    @jit(nopython=True)
    def complex_calculation_serial(n):
        """Cálculo complexo serial"""
        result = 0.0
        for i in range(n):
            result += np.sin(i) * np.cos(i) + np.sqrt(i + 1)
        return result
    
    @jit(nopython=True, parallel=True)
    def complex_calculation_parallel(n):
        """Cálculo complexo paralelo"""
        result = 0.0
        for i in prange(n):
            result += np.sin(i) * np.cos(i) + np.sqrt(i + 1)
        return result
    
    print(f"\\n🧮 Cálculo complexo (n = 10M):")
    n = 10_000_000
    
    # Serial
    start = time.perf_counter()
    result_serial = complex_calculation_serial(n)
    time_serial = time.perf_counter() - start
    
    # Paralelo
    start = time.perf_counter()
    result_parallel = complex_calculation_parallel(n)
    time_parallel = time.perf_counter() - start
    
    speedup = time_serial / time_parallel
    print(f"  Serial:   {time_serial:.4f}s, resultado = {result_serial:.6f}")
    print(f"  Paralelo: {time_parallel:.4f}s, resultado = {result_parallel:.6f}")
    print(f"  Speedup:  {speedup:.2f}x")
    
else:
    print("⚠️  Numba não está disponível. Instale com: pip install numba")
    print("   Numba oferece speedups de 10-100x para código NumPy/Python!")

## 5. Análise de Escalabilidade

Vamos estudar como a performance varia com o tamanho do problema e número de recursos.

In [ ]:
def vector_operation_worker(args):
    """Worker para operações vetoriais"""
    start_idx, end_idx, data = args
    # Simular operação complexa: múltiplas operações matemáticas
    result = np.sum(data[start_idx:end_idx]**2) + np.sum(np.sin(data[start_idx:end_idx]))
    return result

def scaling_study_strong(vector_size, max_processes):
    """
    Strong scaling: tamanho fixo, aumentar processos
    """
    print(f"\\n📊 Strong Scaling - Tamanho fixo: {vector_size:,} elementos")
    print("-" * 60)
    
    # Gerar dados
    np.random.seed(42)
    data = np.random.randn(vector_size)
    
    results = []
    
    for n_proc in range(1, max_processes + 1):
        # Dividir trabalho
        chunk_size = vector_size // n_proc
        
        start_time = time.perf_counter()
        
        if n_proc == 1:
            # Serial
            total = vector_operation_worker((0, vector_size, data))
        else:
            # Paralelo
            with ProcessPoolExecutor(max_workers=n_proc) as executor:
                futures = []
                for i in range(n_proc):
                    start_idx = i * chunk_size
                    end_idx = start_idx + chunk_size if i < n_proc - 1 else vector_size
                    future = executor.submit(vector_operation_worker, (start_idx, end_idx, data))
                    futures.append(future)
                
                total = sum(future.result() for future in as_completed(futures))
        
        exec_time = time.perf_counter() - start_time
        
        if n_proc == 1:
            serial_time = exec_time
            speedup = 1.0
            efficiency = 1.0
        else:
            speedup = serial_time / exec_time
            efficiency = speedup / n_proc
        
        results.append({
            'processes': n_proc,
            'time': exec_time,
            'speedup': speedup,
            'efficiency': efficiency
        })
        
        print(f"  {n_proc:2d} processos: {exec_time:.4f}s, speedup: {speedup:.2f}x, eficiência: {efficiency:.2f}")
    
    return results

def scaling_study_weak(base_size_per_process, max_processes):
    """
    Weak scaling: trabalho por processo fixo, aumentar processos
    """
    print(f"\\n📈 Weak Scaling - {base_size_per_process:,} elementos por processo")
    print("-" * 60)
    
    results = []
    serial_time = None
    
    for n_proc in range(1, max_processes + 1):
        vector_size = base_size_per_process * n_proc
        
        # Gerar dados
        np.random.seed(42)
        data = np.random.randn(vector_size)
        
        start_time = time.perf_counter()
        
        if n_proc == 1:
            # Serial
            total = vector_operation_worker((0, vector_size, data))
            serial_time = time.perf_counter() - start_time
            speedup = 1.0
            efficiency = 1.0
        else:
            # Paralelo
            chunk_size = vector_size // n_proc
            
            with ProcessPoolExecutor(max_workers=n_proc) as executor:
                futures = []
                for i in range(n_proc):
                    start_idx = i * chunk_size
                    end_idx = start_idx + chunk_size if i < n_proc - 1 else vector_size
                    future = executor.submit(vector_operation_worker, (start_idx, end_idx, data))
                    futures.append(future)
                
                total = sum(future.result() for future in as_completed(futures))
            
            exec_time = time.perf_counter() - start_time
            speedup = (serial_time * n_proc) / exec_time  # Speedup para weak scaling
            efficiency = speedup / n_proc
        
        if n_proc > 1:
            exec_time = time.perf_counter() - start_time
        else:
            exec_time = serial_time
        
        results.append({
            'processes': n_proc,
            'problem_size': vector_size,
            'time': exec_time,
            'speedup': speedup,
            'efficiency': efficiency
        })
        
        print(f"  {n_proc:2d} processos ({vector_size:7,} elementos): {exec_time:.4f}s, eficiência: {efficiency:.2f}")
    
    return results

# Executar estudos de escalabilidade
print("🔬 Exemplo 8: Análise de Escalabilidade")
print("=" * 50)

max_proc = min(mp.cpu_count(), 8)  # Limitar para visualização

# Strong scaling
strong_results = scaling_study_strong(1_000_000, max_proc)

# Weak scaling
weak_results = scaling_study_weak(250_000, max_proc)

# Visualização
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Strong scaling - Speedup
processes = [r['processes'] for r in strong_results]
speedups = [r['speedup'] for r in strong_results]
ideal_speedup = processes

ax1.plot(processes, speedups, 'bo-', label='Speedup Real', linewidth=2, markersize=8)
ax1.plot(processes, ideal_speedup, 'r--', label='Speedup Ideal', linewidth=2)
ax1.set_xlabel('Número de Processos')
ax1.set_ylabel('Speedup')
ax1.set_title('Strong Scaling - Speedup')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xticks(processes)

# Strong scaling - Eficiência
efficiencies = [r['efficiency'] for r in strong_results]

ax2.plot(processes, efficiencies, 'go-', linewidth=2, markersize=8)
ax2.axhline(y=1.0, color='r', linestyle='--', alpha=0.7)
ax2.set_xlabel('Número de Processos')
ax2.set_ylabel('Eficiência')
ax2.set_title('Strong Scaling - Eficiência')
ax2.grid(True, alpha=0.3)
ax2.set_xticks(processes)
ax2.set_ylim(0, 1.1)

# Weak scaling - Tempo
weak_processes = [r['processes'] for r in weak_results]
weak_times = [r['time'] for r in weak_results]

ax3.plot(weak_processes, weak_times, 'mo-', linewidth=2, markersize=8)
ax3.axhline(y=weak_times[0], color='r', linestyle='--', alpha=0.7, label='Tempo Ideal')
ax3.set_xlabel('Número de Processos')
ax3.set_ylabel('Tempo de Execução (s)')
ax3.set_title('Weak Scaling - Tempo')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_xticks(weak_processes)

# Weak scaling - Eficiência
weak_efficiencies = [r['efficiency'] for r in weak_results]

ax4.plot(weak_processes, weak_efficiencies, 'co-', linewidth=2, markersize=8)
ax4.axhline(y=1.0, color='r', linestyle='--', alpha=0.7)
ax4.set_xlabel('Número de Processos')
ax4.set_ylabel('Eficiência')
ax4.set_title('Weak Scaling - Eficiência')
ax4.grid(True, alpha=0.3)
ax4.set_xticks(weak_processes)
ax4.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

print("\\n📋 Interpretação dos Resultados:")
print("• Strong scaling: performance com problema fixo")
print("• Weak scaling: performance com trabalho/processo fixo")
print("• Eficiência ideal = 1.0 (100%)")
print("• Overhead reduz eficiência para problemas pequenos")
print("• Importante para dimensionar clusters e escolher algoritmos")

## 6. Resumo e Próximos Passos

### ✅ O que aprendemos hoje:

1. **Ferramentas de medição de performance**
   - Decorators para timing
   - cProfile para análise detalhada

2. **Concurrent.futures avançado**
   - ProcessPoolExecutor com futures
   - Integração numérica paralela

3. **Biblioteca Joblib**
   - Interface simples para paralelização
   - Ideal para análises paramétricas

4. **Numba para aceleração extrema**
   - Compilação JIT (Just-In-Time)
   - Paralelismo automático com `prange`
   - Speedups de 10-100x

5. **Análise de escalabilidade**
   - Strong vs Weak scaling
   - Medição de eficiência

### 🚀 Próxima aula:
- **Computação GPU** com CuPy e Numba CUDA
- **Paralelismo massivo** (milhares de cores)
- **Simulação de difusão de calor** 2D
- **Comparação CPU vs GPU**

### 📝 Tarefa para casa:
Testar escalabilidade de um código próprio e comparar diferentes ferramentas de paralelização.